
# **What is the CBOW Model?**

The CBOW model tries to understand the context of the words and takes this as input. It then tries to predict words that are contextually accurate. Let us consider an example for understanding this. Consider the sentence: ‘It is a pleasant day’ and the word ‘pleasant’ goes as input to the neural network. We are trying to predict the word ‘day’ here. We will use the one-hot encoding for the input words and measure the error rates with the one-hot encoded target word. Doing this will help us predict the output based on the word with least error. 

## **Implementation of the CBOW Model**

For the implementation of this model, we will use a sample text data about coronavirus. You can use any text data of your choice. But to use the data sample I have used [click here](https://github.com/bhoomikamadhukar/NLP) to download the data.

Now that you have the data ready, let us import the libraries and read our dataset. 

In [2]:
!python -m pip install pip --upgrade --user -q --no-warn-script-location
!python -m pip install numpy pandas seaborn matplotlib scipy statsmodels sklearn nltk gensim --user -q --no-warn-script-location


In [3]:
import IPython
IPython.Application.instance().kernel.do_shutdown(True)

{'status': 'ok', 'restart': True}

In [ ]:
import numpy as np
import keras.backend as K
from keras.models import Sequential
from keras.layers import Dense, Embedding, Lambda
from keras.utils import np_utils
from keras.preprocessing import sequence
from keras.preprocessing.text import Tokenizer
import gensim
data=open('corona.txt','r')
corona_data = [text for text in data if text.count(' ') >= 2]
vectorize = Tokenizer()
vectorize.fit_on_texts(corona_data)
corona_data = vectorize.texts_to_sequences(corona_data)
total_vocab = sum(len(s) for s in corona_data)
word_count = len(vectorize.word_index) + 1
window_size = 2

Error: Session cannot generate requests

In the above code, I have also used the built-in method to tokenize every word in the dataset and fit our data to the tokenizer. Once that is done, we need to calculate the total number of words and the total number of sentences as well for further use. As mentioned in the model architecture, we need to assign the window size and I have assigned it to 2.

The next step is to write a function that generates pairs of the context words and the target words. The function below does exactly that. Here we have generated a function that takes in window sizes separately for target and the context and creates the pairs of contextual words and target words. 

In [ ]:
def cbow_model(data, window_size, total_vocab):
  total_length = window_size*2
  for text in data:
      text_len = len(text)
      for idx, word in enumerate(text):
          context_word = []
          target   = []            
          begin = idx - window_size
          end = idx + window_size + 1
          context_word.append([text[i] for i in range(begin, end) if 0 <= i < text_len and i != idx])
          target.append(word)
          contextual = sequence.pad_sequences(context_word, total_length=total_length)
          final_target = np_utils.to_categorical(target, total_vocab)
          yield(contextual, final_target) 

Error: Session cannot generate requests

Finally, it is time to build the neural network model that will train the CBOW on our sample data.

In [ ]:
model = Sequential()
model.add(Embedding(input_dim=total_vocab, output_dim=100, input_length=window_size*2))
model.add(Lambda(lambda x: K.mean(x, axis=1), output_shape=(100,)))
model.add(Dense(total_vocab, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam')
for i in range(10):
    cost = 0
    for x, y in cbow_model(data, window_size, total_vocab):
        cost += model.train_on_batch(contextual, final_target)
    print(i, cost)

Error: Session cannot generate requests

Once we have completed the training its time to see how the model has performed and test it on some words. But to do this, we need to create a file that contains all the vectors. Later we can access these vectors using the gensim library. 

In [ ]:
dimensions=100
vect_file = open('vectors.txt' ,'w')
vect_file.write('{} {}\n'.format(total_vocab,dimensions))

Error: Session cannot generate requests

Next, we will access the weights of the trained model and write it to the above created file.

In [ ]:
weights = model.get_weights()[0]
for text, i in vectorize.word_index.items():
    final_vec = ' '.join(map(str, list(weights[i, :])))
    vect_file.write('{} {}\n'.format(text, final_vec))
vect_file.close()

Now we will use the vectors that were created and use them in the gensim model. The word I have chosen in ‘virus’.

In [1]:
cbow_output = gensim.models.KeyedVectors.load_word2vec_format('vectors.txt')
cbow_output.most_similar(positive=['virus'])

NameError: name 'gensim' is not defined

# **Read more articles on:**

> * [Continuous Bag of Words](https://analyticsindiamag.com/the-continuous-bag-of-words-cbow-model-in-nlp-hands-on-implementation-with-codes/)

> * [NLP Case Study of Documents Similarity](https://analyticsindiamag.com/nlp-case-study-identify/)

> * [Review Classification](https://analyticsindiamag.com/step-by-step-guide-to-reviews-classification-using-svc-naive-bayes-random-forest/)

> * [Multi Class Text Classification](https://analyticsindiamag.com/multi-class-text-classification-in-pytorch-using-torchtext/)

> * [Text Classification](https://analyticsindiamag.com/how-to-solve-your-first-ever-nlp-classification-challenge/)

> * [Word Frequency](https://analyticsindiamag.com/using-natural-language-processing-to-check-word-frequency-in-the-adventure-of-sherlock-holmes/)

